
# 06 — RAG Architecture Diagrams  
RAG v1 • RAG v2 • GraphRAG • Agentic RAG (LangGraph)

## 1. RAG v1 — Basic Pipeline
RAG v1 is the simplest retrieval-augmented generation pipeline.


In [ ]:

!pip install langchain langchain-openai chromadb langchain-community --quiet

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

docs = [TextLoader("sample_doc.txt").load()[0]]
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)
emb = OpenAIEmbeddings()
db = Chroma.from_documents(chunks, emb)

def rag_v1(question):
    retr = db.similarity_search(question, k=4)
    ctx = "\n\n".join([x.page_content for x in retr])
    prompt = f"Context:\n{ctx}\n\nQ: {question}\nA:"
    llm = ChatOpenAI(model="gpt-4o-mini")
    return llm.invoke(prompt).content

rag_v1("What does the document say about training data?")


In [ ]:

from rank_bm25 import BM25Okapi
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

corpus = [c.page_content.split() for c in chunks]
bm25 = BM25Okapi(corpus)

def hybrid_retrieve(question, top_k=8):
    vec_docs = db.similarity_search(question, k=top_k)
    bm25_scores = bm25.get_scores(question.split())
    bm_docs = sorted(zip(chunks, bm25_scores), key=lambda x: x[1], reverse=True)[:top_k]
    merged = list({d.page_content: d for d in vec_docs}.values())
    merged.extend([x[0] for x in bm_docs])
    return merged[:top_k]

def rerank(question, retrieved_docs):
    llm = ChatOpenAI(model="gpt-4o-mini")
    scored = []
    for d in retrieved_docs:
        prompt = f"Score relevance 0-10.\nQ: {question}\nDoc: {d.page_content[:500]}\nScore:"
        s = llm.invoke(prompt).content.strip()
        try: score = float(s)
        except: score = 0
        scored.append((d, score))
    return [x[0] for x in sorted(scored, key=lambda x: x[1], reverse=True)]

def rag_v2(question):
    retrieved = hybrid_retrieve(question)
    reranked = rerank(question, retrieved)
    ctx = "\n\n".join([r.page_content for r in reranked[:5]])
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt = f"Context:\n{ctx}\n\nQ:{question}\nA:"
    return llm.invoke(prompt).content

rag_v2("Summarize the training process.")


In [ ]:

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

emb_vectors = emb.embed_documents([c.page_content for c in chunks])
sim_matrix = cosine_similarity(emb_vectors)
threshold = 0.70
edges = {i: [j for j, sim in enumerate(row) if sim > threshold and i != j] 
         for i, row in enumerate(sim_matrix)}

def graph_retrieve(question, hops=2):
    qv = emb.embed_query(question)
    sims = cosine_similarity([qv], emb_vectors)[0]
    start = int(np.argmax(sims))
    visited = set([start])
    frontier = [start]
    for _ in range(hops):
        new_nodes = []
        for n in frontier:
            for neigh in edges[n]:
                if neigh not in visited:
                    visited.add(neigh)
                    new_nodes.append(neigh)
        frontier = new_nodes
    return [chunks[i] for i in visited]

def graph_rag(question):
    docs = graph_retrieve(question)
    ctx = "\n\n".join([d.page_content for d in docs])
    llm = ChatOpenAI(model="gpt-4o-mini")
    return llm.invoke(f"Context:\n{ctx}\n\nQ:{question}\nA:").content

graph_rag("Explain the concept of data processing.")


In [ ]:

!pip install langgraph --quiet
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

class RAGState(TypedDict):
    question: str
    plan: str
    context: str
    answer: str

llm = ChatOpenAI(model="gpt-4o-mini")

def plan_node(state: RAGState):
    out = llm.invoke([HumanMessage(content=f"Break into search plan:\n{state['question']}")])
    state["plan"] = out.content
    return state

def retrieve_node(state: RAGState):
    docs = db.similarity_search(state["plan"], k=4)
    state["context"] = "\n\n".join([d.page_content for d in docs])
    return state

def answer_node(state: RAGState):
    prompt = f"Context:\n{state['context']}\n\nQ:{state['question']}\nA:"
    out = llm.invoke(prompt)
    state["answer"] = out.content
    return state

g = StateGraph(RAGState)
g.add_node("plan", plan_node)
g.add_node("retrieve", retrieve_node)
g.add_node("answer", answer_node)
g.set_entry_point("plan")
g.add_edge("plan", "retrieve")
g.add_edge("retrieve", "answer")
g.add_edge("answer", END)

agentic_rag = g.compile()
agentic_rag.invoke({"question": "What is the model training workflow?"})
